In [1]:
import rasterio 
from rasterio.mask import raster_geometry_mask,mask
import geopandas as gpd
import os
import numpy as np
import pandas as pd
from pathlib import Path
import json

In [2]:
year = "2017"
phase = "test"

data_dir = "/home/jovyan/work/satellite_data/ku_sync/South_Africa/all"
cutlines = "/home/jovyan/work/satellite_data/ku_sync/South_Africa/cutlines/"+ year+".geojson"
tmp_path = "/home/jovyan/work/satellite_data/tmp"
cut_df = gpd.read_file(cutlines,driver='GeoJSON')

max_raster_size = 2000*2000

In [3]:
path = year + "/" + phase
Path("out",path,"raster").mkdir(parents=True, exist_ok=True)

# Cut out raster from areas

In [4]:
p = os.path.join("area_shapes",path)
area_shapes = sorted([os.path.join(os.getcwd(),p,i) for i in os.listdir(p) if i.endswith(".geojson")])

In [5]:
# gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in area_shapes], 
#                         ignore_index=True), crs=gpd.read_file(area_shapes[0]).crs)

DF = pd.DataFrame({'geometry' : [],"file":[]})
for i in area_shapes:
    df = gpd.read_file(i)
    df["file"] = i
    DF = pd.concat([DF,df], 
                        ignore_index=True)
gdf = gpd.GeoDataFrame(DF, crs=gpd.read_file(area_shapes[0]).crs)

gdf = gdf.drop_duplicates("geometry").reset_index(drop=True)
if "FID" in gdf:
    gdf = gdf.drop('FID', axis=1)

In [6]:
from shapely.geometry import Polygon

def split_raster(poly,raster,ncols=2,nrows=2):
    xmin, ymin, xmax, ymax = poly.bounds

    width = xmax-xmin
    height = ymax-ymin

    wide = width / ncols
    length = height / nrows

    x = xmin
    y = ymin
    polygons = []
    for r in range(nrows):
        if r== nrows-1:
            ystep = length#+(height % nrows)
        else:
            ystep = length

        for c in range(ncols):        
            if (c % ncols) == 0:
                x = xmin
            if c == ncols-1:
                xstep = wide#+(width % ncols)
            else:
                xstep = wide
            polygons.append(Polygon([(x,y), (x+xstep, y), (x+xstep, y+ystep), (x, y+ystep)]))
            x += xstep
        y += ystep


    grid = gpd.GeoDataFrame({'geometry':polygons})
    grid.crs = src.crs

    out = []
    for i in range(len(grid)):
        bounds = grid.iloc[i]["geometry"]
        out_image, out_transform = mask(raster, [bounds], crop=True)
        out_meta = src.meta
        out_meta.update({"driver": "GTiff",
                 "height": out_image.shape[1],
                 "width": out_image.shape[2],
                 "transform": out_transform,
                 'crs':'EPSG:4326'})
        out.append([out_image,out_meta])
    return out

In [ ]:
mapping = {}
for i,s in gdf.iterrows():

        
    file = os.path.basename(s["file"]).split(".")[0]
    poly = s["geometry"]
    idxs = np.where((cut_df["geometry"].covers(poly)) )[0] 
    
    print(i)
    print(s["file"])
    print(idxs)
    if len(idxs) > 1:
        idx = idxs[0]
    elif len(idxs) == 0:
        print("No intersection for ",i)
        continue
    else:
        idx = int(idxs)
    
    p_name = cut_df.iloc[idx]["id"]
    p_area = p_name[:2]
    p_path = os.path.join(data_dir,p_name+"_"+year+".tif")
    if not os.path.isfile(p_path):
        p_path = os.path.join(data_dir,p_name+"_"+year+".jp2")
        
        if not os.path.isfile(p_path):
            p_path = os.path.join(data_dir,p_name+"_"+year+".jp2.tif")
    
    # if not os.path.isfile(p_path):
    #     print(p_path)
    #     p_path = os.path.join(tmp_path,p_area,year+"_cog.tif")
    
    #Workaround for se labels:
    if "se" in s["file"]:
        print("SE")
        p_path = os.path.join(tmp_path,p_area,year+"_cog.tif")
    
    with rasterio.open(p_path) as src:
        arr,transform = mask(src, [poly], crop=True)
        arr.transpose((1,2,0))
        
        if ((np.sum(arr == 0) + np.sum(arr == 255)) > np.prod(arr.shape) * 0.1) and (len(idxs) > 1):
            idx = idxs[1]
            p_name = cut_df.iloc[idx]["id"]
            p_area = p_name[:2]
            p_path = os.path.join(data_dir,p_name+"_"+year+".tif")
            
            with rasterio.open(p_path) as src:
                arr,transform = mask(src, [poly], crop=True)
                arr.transpose((1,2,0))
    
        
        if np.prod(arr.shape) > max_raster_size:
            arr_list = split_raster(poly,src)
            
            for n,data in enumerate(arr_list):
                new_file = os.path.join("out",path,"raster",file+"_"+str(i)+"_"+str(n)+".tif")
                out_img,out_meta = data
                with rasterio.open(new_file,mode='w',**out_meta) as dst:
                    dst.write(out_img)
        else:
            new_file = os.path.join("out",path,"raster",file+"_"+str(i)+".tif")
            out_meta = src.meta
            out_meta.update({'width':arr.shape[2],
                     'height':arr.shape[1],
                     'transform':transform,
                    'crs':'EPSG:4326',
                     "driver": "GTiff" })
            with rasterio.open(new_file,mode='w',**out_meta) as dst:
                dst.write(arr)
            
        mapping[os.path.basename(s["file"])] = os.path.basename(new_file)
        
    print("#############")
with open(os.path.join("out",path,'mapping.json'), 'w') as fp:
    json.dump(mapping, fp)

0
/home/jovyan/work/notebooks/satellite_data/utils/preprocess/area_shapes/2017/test/2017.geojson
[44]
#############
1
/home/jovyan/work/notebooks/satellite_data/utils/preprocess/area_shapes/2017/test/2017.geojson
[337]


# Merge different label files

In [ ]:
p = os.path.join("labels",path)
label_files = sorted([os.path.join(os.getcwd(),p,i) for i in os.listdir(p) if i.endswith(".geojson")])

In [ ]:
gdf_labels = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in label_files], 
                        ignore_index=True), crs=gpd.read_file(area_shapes[0]).crs)
gdf_labels.to_file(os.path.join("out",path,"labels.geojson"))

# Old

In [ ]:
stop


In [ ]:
#from https://stackoverflow.com/questions/5953373/how-to-split-image-into-multiple-pieces-in-python
def split_array(im):
    M = int(np.ceil(im.shape[1]/2))
    N = int(np.ceil(im.shape[2]/2))
    
    tiles = [im[:,x:x+M,y:y+N] for x in range(0,im.shape[1],M) for y in range(0,im.shape[2],N)]
    
    return tiles